# Tutorial: Introduction to modeling in Gen

This tutorial introduces Gen's built-in modeling language, and illustrates probabilistic inference in Gen using a simple generic inference algorithm.

This tutorial will guide you through how to:

- Express a probabilistic model as a generative function in Gen

- Obtain the trace of a generative function, and inspect and visualize it the trace.

- Write a simple inference program based on a generic importance sampling inference algorithm.

- Interpret the output of the inference program, and the effect of amount of computation on accuracy of inferences.

The tutorial also illustrates different types of modeling flexibility afforded by the built-in modeling language. You will:

- Write a probabilistic model that uses a stochastic if-else branch to infer which model of two models best explains a data set.

- Write a probabilistic model that uses an unbounded number of parameters using recursion.

Note that this tutorial does not cover *inference programming*, in which users implement inference algorithms that are specialized to their probabilistic model. Inference programming is important for getting accurate inferences efficiently, and will be covered in later tutorials. Also, this tutorial does not exhaustively cover all features of the modeling language -- there are also various features and extensions that provide improved performance that are not covered here.

## Outline

**Section 1.** [Julia, Gen, and this Jupyter notebook](#julia-gen-jupyter)

**Section 2.** [Writing a probabilistic model as a generative function](#writing-model)

**Section 3.** [Doing Bayesian inference](#doing-inference)

**Section 4.** [Predicting new data](#predicting-data)

**Section 5.** [Calling other generative functions](#calling-functions)

**Section 6.** [Modeling with an infinite discrete hypothesis space](#infinite-space)

## 1. Julia, Gen, and this Jupyter notebook  <a name="julia-gen-jupyter"></a>

Gen is a package for the Julia language. The package can be loaded with:

In [ ]:
using Gen

Gen programs typically consist of a combination of (i) probabilistic models written in modeling languages and (ii) inference programs written in regular Julia code. Gen provides a built-in modeling language that is itself based on Julia.

This tutorial uses a Jupyter notebook. All cells in the notebook are regular Julia cells. Throughout the tutorial, we will use  that we use semicolons at the end of some cells so that the value of a cell is not printed.

In [ ]:
a = 1 + 1

In [ ]:
a = 1 + 1;

This notebook uses the [PyPlot](https://github.com/JuliaPy/PyPlot.jl) Julia package for plotting. PyPlot wraps the matplotlib Python package.

In [ ]:
using PyPlot

This notebook will make use of Julia symbols. Note that a Julia symbol is different from a Julia string:

In [ ]:
typeof(:foo)

In [ ]:
typeof("foo")

## 2. Writing a probabilistic model as a generative function  <a name="writing-model"></a>

Probabilistic models represented in Gen as *generative functions*. The simplest way to construct a generative function is by using the built-in modeling DSL. Generative functions written in the built-in modeling DSL are based on Julia function definition syntax, but are prefixed with the `@gen` keyword. The generative function below represents a probabilistic model of a line in the x-y plane, and values of the y-coordinates associated with a given set of x-coordinates.

In [ ]:
@gen function line_model(xs::Vector{Float64})
    n = length(xs)
    slope = @addr(normal(0, 1), :slope)
    intercept = @addr(normal(0, 2), :intercept)
    for (i, x) in enumerate(xs)
        @addr(normal(slope * x + intercept, 0.1), (:y, i))
    end
    return n
end;

The generative function takes as an argument a vector of x-coordinates. We create one below:

In [ ]:
xs = [-5., -4., -3., -.2, -1., 0., 1., 2., 3., 4., 5.];

The generative function then samples a random choice representing the slope of a line from a normal distribution with mean 0 and standard deviation 1, and a random choice representing the intercept of a line from a normal distribution with mean 0 and standard deviation 2. In Bayesian statistics terms, these distributions are the *prior distributions* of the slope and intercept respectively. Then, the function samples values for the y-coordinates corresponding to each of the provided x-coordinates Each random choice has a unique *address*. A random choice is assigned an address using the `@addr` keyword. Addresses can be any Julia value. In this program, there are two types of addresses used -- Julia symbols and tuples of symbols and integers.

-------------------------------------------------
### Exercise
List the addresses of all random choices made when applying `line_model` to the vector `xs` defined above.

-------------------------------------------------
### Exercise

Write a generative function that uses the same address twice. Run it to see what happens.

-------------------------------------------------

This generative function returns the number of data points. We can run the function like we run a regular Julia function:

In [ ]:
n = line_model(xs)
println(n)

It is the random choices made by this generative function that are most important. The random choices are not included in the return value. They are however, included in the *trace* of the generative function. We can run the generative function and obtain its trace using the a method from the Gen API:

In [ ]:
(trace, _) = Gen.initialize(line_model, (xs,));

This method takes the function to be executed, and a tuple of arguments to the function, and returns a trace and a second value that we will not be using in this tutorial. When we print the trace, we see that it is a complex data structure.

In [ ]:
println(trace)

The trace contains various data about the execution. In particular, in contains the arguments on which the function was run, which are available with an API method:

In [ ]:
Gen.get_args(trace)

The trace also contains the value of the random choices, stored in map from addresses to their values. This map is also available through an API method:

In [ ]:
println(Gen.get_assmt(trace))

The return value is also recorded in the trace:

In [ ]:
println(Gen.get_retval(trace));

In order to understand the probabilistic behavior of a generative function, it is helpful to be able to visualize the trace of a generative function. Below, we define a function that uses PyPlot to render a trace of the generative function above. The rendering shows the x-y data points and the line that is represented by the slope and intercept choices.

In [ ]:
function render_trace(trace; show_data=true)
    xs = get_args(trace)[1]
    assmt = get_assmt(trace)
    if show_data
        ys = [assmt[(:y, i)] for i=1:length(xs)]
        scatter(xs, ys, c="black")
    end
    slope = assmt[:slope]
    intercept = assmt[:intercept]
    xmin = minimum(xs)
    xmax = maximum(xs)
    plot([xmin, xmax], slope *  [xmin, xmax] .+ intercept, color="black", alpha=0.5)
    ax = gca()
    ax[:set_xlim]((xmin, xmax))
    ax[:set_ylim]((xmin, xmax))
end;

In [ ]:
figure(figsize=(3,3))
render_trace(trace);

Because a generative function is stochastic, we need to visualize many runs in order to understand its behavoir. The cell below will allow us to render a grid of traces.

In [ ]:
function grid(renderer, traces; ncols=6, nrows=3)
    figure(figsize=(16, 8))
    for (i, trace) in enumerate(traces)
        subplot(nrows, ncols, i)
        renderer(trace)
    end
end;

Now, we generate several traces and render them in a grid

In [ ]:
traces = [initialize(line_model, (xs,))[1] for _=1:12]
grid(render_trace, traces)

-------------------------------
### Exercise

Write a model that generates a sine wave of unknown phase, period and amplitude, and then generates y-coordinates from a given vector of x-coordinates by adding noise to the value of the wave at each x-coordinate.
Use a Gamma distribution  (see [`Gen.gamma`](https://probcomp.github.io/Gen/dev/ref/distributions/#Gen.gamma)). for the prior distributions on the period and amplitude, and a uniform distribution for the phase (see [`Gen.uniform`](https://probcomp.github.io/Gen/dev/ref/distributions/#Gen.uniform)). Write a function that renders the trace by showing the data set and the sine wave. Visualize a grid of traces and discuss the distribution. Try tweaking the parameters of each of the prior distributions and seeing how the behavior changes.

Hint: There should be three random choices corresponding to the period, amplitude, and phase, and then N random choices, one for each y-coordinate.

## 3. Doing Bayesian inference  <a name="doing-inference"></a>

We now will provide a data set of y-coordinates and try to draw inferences about the process that generated the data. We begin with the following data set:

In [ ]:
ys = [6.75003, 6.1568, 4.26414, 1.84894, 3.09686, 1.94026, 1.36411, -0.83959, -0.976, -1.93363, -2.91303];

In [ ]:
figure(figsize=(3,3))
scatter(xs, ys, color="black");

We will start by assuming that the line model was responsible for generatin the data, and inferring values of the slope and intercept that explain the data.

To do this, we write a simple *inference program* that takes the model we are assuming, the data set, and the amount of computation to perform, and returns a trace of the function that is approximately sampled from the posterior distribution on traces of the function, given the observed data. This inference program is based on a Gen API method `importance_resampling`. Don't worry about the internals of this inference program yet. We will discuss inference programming in later tutorials.

In [ ]:
function do_inference(model, xs, ys, amount_of_computation)
    observations = Gen.DynamicAssignment()
    for (i, y) in enumerate(ys)
        observations[(:y, i)] = y
    end
    (trace, _) = Gen.importance_resampling(model, (xs,), observations, amount_of_computation);
    return trace
end;

We run the inference program and visualize the result.

In [ ]:
trace = do_inference(line_model, xs, ys, 100)
figure(figsize=(3,3))
render_trace(trace);

We can also visualize many samples in a grid.

In [ ]:
traces = [do_inference(line_model, xs, ys, 100) for _=1:10];
grid(render_trace, traces)

However, in this case we can get a better sense for the variability in the posterior distribution by overlaying the traces. Each trace is going to have the same observed data points, so we only plot those once, based on the values in the first trace.

In [ ]:
function overlay(renderer, traces; same_data=true, args...)
    renderer(traces[1], show_data=true, args...)
    for i=2:length(traces)
        renderer(traces[i], show_data=!same_data, args...)
    end
end;

In [ ]:
traces = [do_inference(line_model, xs, ys, 100) for _=1:10];
figure(figsize=(3,3))
overlay(render_trace, traces);

--------------

### Exercise

The results above were obtained for `amount_of_computation = 100`. Run the algorithm with this value set to `1`, `10`, and `1000`, etc.  Which value seems like a good tradeoff between accuracy and running time? Discuss.

------------------

### Exercise
Consider the following data set.

In [ ]:
ys_sine = [2.89, 2.22, -0.612, -0.522, -2.65, -0.133, 2.70, 2.77, 0.425, -2.11, -2.76];

In [ ]:
figure(figsize=(3, 3));
scatter(xs, ys_sine, color="black");

Write an inference program that generates traces of the sine wave model that explain this data set. Visualize the resulting distribution of traces. Experiment with a `gamma(1, 1)` and `gamma(5, 1)` prior on the period. Read about the Gamma distribution at https://en.wikipedia.org/wiki/Gamma_distribution. Discuss the results of inference? Do they make sense? How much computation did you need to get good results?

## 4. Predicting new data  <a name="predicting-data"></a>

By providing a third argument to `Gen.initialize`, it is possible to run a generatiev function with the values of certain random choices constrained to given values. The third argument an assignment. For example:

In [ ]:
constraints = Gen.DynamicAssignment()
constraints[:slope] = 0.
constraints[:intercept] = 0.
(trace, _) = Gen.initialize(line_model, (xs,), constraints)
figure(figsize=(3,3))
render_trace(trace);

Note that the random choices corresponding to the y-coordinates are still made randomly. Run the cell above a few times to verify this.

We will use the ability to run constrained executions of a generative function to predict the value of the y-coordinates at new x-coordinates by running new executions of the model generative function in which the random choices corresponding to the parameters have been constrained to their inferred values.  We have provided a function below that takes a trace, and a vector of new x-coordinates, and returns a vector of predicted y-coordinates corresponding to the x-coordinates in `new_xs`. We have designed this function to work with multiple models, so the set of parameter addresses is an argument.

In [ ]:
function predict_new_data(model, trace, new_xs::Vector{Float64}, param_addrs)
    constraints = Gen.DynamicAssignment()
    assmt = Gen.get_assmt(trace)
    for addr in param_addrs
        if Gen.has_value(assmt, addr)
            constraints[addr] = assmt[addr]
        end
    end
    (new_trace, _) = Gen.initialize(model, (new_xs,), constraints)
    new_assmt = Gen.get_assmt(new_trace)
    ys = [new_assmt[(:y, i)] for i=1:length(new_xs)]
    return ys
end;

The cell below defines a function that first performs inference on an observed data set `(xs, ys)`, and then runs `predict_new_data` to generate predicted y-coordinates. It repeats this process `num_traces` times, and returns a vector of the resulting y-coordinate vectors.

In [ ]:
function infer_and_predict(model, xs, ys, new_xs, param_addrs, num_traces, amount_of_computation)
    pred_ys = []
    for i=1:num_traces
        trace = do_inference(model, xs, ys, amount_of_computation)
        push!(pred_ys, predict_new_data(model, trace, new_xs, param_addrs))
    end
    pred_ys
end;

Finally, we define a cell that plots the observed data set `(xs, ys)` as red dots, and the predicted data as small black dots.

In [ ]:
function plot_predictions(xs, ys, new_xs, pred_ys)
    scatter(xs, ys, color="red")
    for pred_ys_single in pred_ys
        scatter(new_xs, pred_ys_single, color="black", s=1, alpha=0.3)
    end
end;

Recall the original dataset for the line model. The x-coordinates span the interval -5 to 5.

In [ ]:
figure(figsize=(3,3))
scatter(xs, ys, color="red");

We will use the inferred values of the parameters to predict y-coordinates for x-coordinates in the interval 5 to 10 from which data was not observed. We will also predict new data within the interval -5 to 5, and we will compare this data to the original observed data. Predicting new data from inferred parameters, and comparing this new data to the observed data is the core idea behind *posterior predictive checking*. This tutorial does not intend to give a rigorous overview behind techniques for checking the quality of a model, but intends to give high-level intuition.

In [ ]:
new_xs = collect(range(-5, stop=10, length=100));

We generate and plot the predicted data:

In [ ]:
pred_ys = infer_and_predict(line_model, xs, ys, new_xs, [:slope, :intercept], 20, 1000)
figure(figsize=(3,3))
plot_predictions(xs, ys, new_xs, pred_ys)

The results look reasonable, both within the interval of observed data and in the extrapolated predictions on the right.

Now consider the same experiment run with following data set, which has significantly more noise.

In [ ]:
ys_noisy = [5.092, 4.781, 2.46815, 1.23047, 0.903318, 1.11819, 2.10808, 1.09198, 0.0203789, -2.05068, 2.66031];

In [ ]:
pred_ys = infer_and_predict(line_model, xs, ys_noisy, new_xs, [:slope, :intercept], 20, 1000)
figure(figsize=(3,3))
plot_predictions(xs, ys_noisy, new_xs, pred_ys)

It looks like the generated data is less noisy than the observed data in the regime where data was observed, and it looks like the forecasted data is too overconfident. This is a sign that our model is mis-specified. In our case, this is because we have assumed that the noise has value 0.1. However, the actual noise in the data appears to be much larger. We can correct this by making the noise a random choice as well and inferring its value along with the other parameters.

We first write a new version of the line model that samples a random choice for the noise from a `gamma(1, 1)` prior distribution.

In [ ]:
@gen function line_model_2(xs::Vector{Float64})
    n = length(xs)
    slope = @addr(normal(0, 1), :slope)
    intercept = @addr(normal(0, 2), :intercept)
    noise = @addr(gamma(1, 1), :noise)
    for (i, x) in enumerate(xs)
        @addr(normal(slope * x + intercept, noise), (:y, i))
    end
    return nothing
end;

Then, we compare the predictions using inference the unmodified and modified model on the `ys` data set:

In [ ]:
figure(figsize=(6,3))
subplot(1, 2, 1)
pred_ys = infer_and_predict(line_model, xs, ys, new_xs, [:slope, :intercept], 20, 1000)
plot_predictions(xs, ys, new_xs, pred_ys)
subplot(1, 2, 2)
pred_ys = infer_and_predict(line_model_2, xs, ys, new_xs, [:slope, :intercept, :noise], 20, 10000)
plot_predictions(xs, ys, new_xs, pred_ys)

Notice that there is more uncertainty in the predictions made using the modified model.

We also compare the predictions using inference the unmodified and modified model on the `ys_noisy` data set:

In [ ]:
figure(figsize=(6,3))
subplot(1, 2, 1)
pred_ys = infer_and_predict(line_model, xs, ys_noisy, new_xs, [:slope, :intercept], 20, 1000)
plot_predictions(xs, ys_noisy, new_xs, pred_ys)
subplot(1, 2, 2)
pred_ys = infer_and_predict(line_model_2, xs, ys_noisy, new_xs, [:slope, :intercept, :noise], 20, 10000)
plot_predictions(xs, ys_noisy, new_xs, pred_ys)

Notice that while the unmodified model was very overconfident, the modified model has an appropriate level of uncertainty, while still capturing the general negative trend

-------------------------
### Exercise

Write a modified version the sine model that makes noise into a random choice. Compare the predicted data with the observed data `infer_and_predict` and `plot_predictions` for the unmodified and modified model, and for the `ys_sine` and `ys_noisy` datasets. Discuss the results. Experiment with the amount of inference computation used. The amount of inference computation will need to be higher for the model with the noise random choice. We have provided you with starter code. Convert these cells from `Raw NBConvert` cells to `Code` cells and fill them in.

## 5. Calling other generative functions  <a name="calling-functions"></a>

In addition to making random choices, generative functions can invoke other generative functions. To illustrate this, we will write a probabilistic model that combines the line model and the sine model. This model is able to explain data using either model, and which model is chosen will depend on the data. This is called *model selection*.

A generative function can invoke another generative function in two ways -- using `@splice` or using `@addr`. When invoking using `@splice`, the random choices of the callee function are placed in the same address namespace as the caller's random choices. When using `@addr(<call>, <addr>)`, the random choices of the callee are placed under the namespace `<addr>`.

In [ ]:
@gen function foo()
    @addr(normal(0, 1), :y)
end

@gen function bar_splice()
    @addr(bernoulli(0.5), :x)
    @splice(foo())
end

@gen function bar_addr()
    @addr(bernoulli(0.5), :x)
    @addr(foo(), :z)
end;

We first show the addresses sampled by `bar_splice`:

In [ ]:
(trace, ) = initialize(bar_splice, ())
println(get_assmt(trace))

And the addresses sampled by `bar_addr`:

In [ ]:
(trace, ) = initialize(bar_addr, ())
println(get_assmt(trace))

Using `@addr` instead of `@splice` can help avoid address collisions for complex models.

Now, we write a generative function that combies the line and sine models. It makes a Bernoulli random choice (e.g. a coin flip that returns true or false) that determines which of the two models will generate the data.

In [ ]:
@gen function combined_model(xs::Vector{Float64})
    if @addr(bernoulli(0.5), :is_line)
        @splice(line_model_2(xs))
    else
        @splice(sine_model_3(xs))
    end
end;

We also write a visualization for a trace of this function:

In [ ]:
function render_combined(trace; show_data=true)
    assmt = get_assmt(trace)
    if assmt[:is_line]
        render_trace(trace, show_data=show_data)
    else
        render_sine_trace(trace, show_data=show_data)
    end
end;

We visualize some traces, and see that sometimes it samples linear data and other times sinusoidal data.

In [ ]:
traces = [initialize(combined_model, (xs,))[1] for _=1:12];
grid(render_combined, traces)

We run inference using this combined model on the `ys` data set and the `ys_sine` data set. 

In [ ]:
figure(figsize=(6,3))
subplot(1, 2, 1)
traces = [do_inference(combined_model, xs, ys, 10000) for _=1:10];
overlay(render_combined, traces)
subplot(1, 2, 2)
traces = [do_inference(combined_model, xs, ys_sine, 10000) for _=1:10];
overlay(render_combined, traces)

------
### Exercise 

There is code duplication in `line_model_3` and `sine_model_3`. Refactor the model to reduce code duplication and improve the readability of the code. Re-run the experiment above and confirm that the results are qualitatively the same. You may need to write a new rendering function. Try to avoid introducing code duplication between the model and the rendering code.

Hint: To avoid introducing code duplication between the model and the rendering code, use the return value of the generative function.

-------

### Exercise

Construct a data set for which it is ambiguous whether the line or sine wave model is best. Visualize the inferred traces usingn `render_either` to illustrate the ambiguity. Write a program that takes the data set and returns an estimate of the posterior probability that the data was generated by the sine wave model, and run it on your data set.

Hint: To estimate the posterior probability that the data was generated by the sine wave model, run the inference program many times to compute a large number of traces, and then compute the fraction of those traces in which `:is_line` is false.

## 6. Modeling with an infinite discrete hypothesis space  <a name="infinite-space"></a>

Gen's built-in modeling language can be used to express models that include more complex explanations for data that from hypothesis spaces that include an infinite set of possible discrete combinatorial structures. This section walks you through development of a model of data that does not a-priori specify an upper bound on the complexity model, but instead infers the complexity of the model as well as the parameters. This is a simple example of a *Bayesian nonparametric* model.

We will consider two data sets:

In [ ]:
xs_dense = collect(range(-5, stop=5, length=50))
ys_simple = fill(1., length(xs_dense)) .+ randn(length(xs_dense)) * 0.1
ys_complex = [Int(floor(abs(x/3))) % 2 == 0 ? 2 : 0 for x in xs_dense] .+ randn(length(xs_dense)) * 0.1;

In [ ]:
figure(figsize=(6,3))
subplot(1, 2, 1)
scatter(xs_dense, ys_simple, color="black", s=10)
gca()[:set_ylim]((-1, 3))
subplot(1, 2, 2)
scatter(xs_dense, ys_complex, color="black", s=10);
gca()[:set_ylim]((-1, 3));

The data set on the left appears to be best explained as a contant function with some noise. The data set on the right appears to include four changepoints, with a constant function in between the changepoints. We want a model that does not a-priori choose the number of changepoints in the data. To do this, we will recursively partition the interval into regions.

In [ ]:
struct Interval
    l::Float64
    u::Float64
end

In [ ]:
abstract type Node end
    
struct InternalNode <: Node
    left::Node
    right::Node
    interval::Interval
end

struct LeafNode <: Node
    value::Float64
    interval::Interval
end

In [ ]:
@gen function generate_segments(l::Float64, u::Float64)
    interval = Interval(l, u)
    if @addr(bernoulli(0.7), :isleaf)
        value = @addr(normal(0, 1), :value)
        return LeafNode(value, interval)
    else
        frac = @addr(beta(2, 2), :frac)
        mid  = l + (u - l) * frac
        left = @addr(generate_segments(l, mid), :left)
        right = @addr(generate_segments(mid, u), :right)
        return InternalNode(left, right, interval)
    end
end;

In [ ]:
function render_node(node::LeafNode)
    plot([node.interval.l, node.interval.u], [node.value, node.value])
end

function render_node(node::InternalNode)
    render_node(node.left)
    render_node(node.right)
end;

In [ ]:
function render_segments_trace(trace)
    node = get_retval(trace)
    render_node(node)
    ax = gca()
    ax[:set_xlim]((0, 1))
    ax[:set_ylim]((-3, 3))
end;

We generate 12 traces from this function and visualize them below. We plot the piecewise constant function that was sampled by each run of the generative function. Different constant segments are shown in different colors. Run the cell a few times to get a better sense of the distribution on functions that is represented by the generative function.

In [ ]:
traces = [initialize(generate_segments, (0., 1.))[1] for i=1:12]
grid(render_segments_trace, traces)

Now that we have generative function that generates an unknown partition into segments with values, we write a model that adds noise to the resulting constant functions to generate a data set of y-coordinates. The noise level will be a random choice.

In [ ]:
function get_value_at(x::Float64, node::LeafNode)
    @assert x >= node.interval.l && x <= node.interval.u
    return node.value
end

function get_value_at(x::Float64, node::InternalNode)
    @assert x >= node.interval.l && x <= node.interval.u
    if x <= node.left.interval.u
        get_value_at(x, node.left)
    else
        get_value_at(x, node.right)
    end
end

@gen function changepoint_model(xs::Vector{Float64})
    node = @addr(generate_segments(minimum(xs), maximum(xs)), :tree)
    noise = @addr(gamma(1, 1), :noise)
    for (i, x) in enumerate(xs)
        @addr(normal(get_value_at(x, node), noise), (:y, i))
    end
    return node
end;

We write a visualization for `changepoint_model` below:

In [ ]:
function render_cp_model_trace(trace; show_data=true)
    xs = get_args(trace)[1]
    node = get_retval(trace)
    render_node(node)
    assmt = get_assmt(trace)
    if show_data
        ys = [assmt[(:y, i)] for i=1:length(xs)]
        scatter(xs, ys, c="black")
    end
    ax = gca()
    ax[:set_xlim]((minimum(xs), maximum(xs)))
    ax[:set_ylim]((-3, 3))
end;

Finally, we generate some simulated data sets and visualize them on top of the underlying piecewise constant function from which they were generated:

In [ ]:
traces = [initialize(changepoint_model, (xs_dense,))[1] for i=1:12]
grid(render_cp_model_trace, traces)

Notice that the amount of variability around the piecewise constant mean function differs from trace to trace.

Now we perform inference for the simple data set:

In [ ]:
traces = [do_inference(changepoint_model, xs_dense, ys_simple, 10000) for _=1:12];
grid(render_cp_model_trace, traces)

We see that we inferred that the mean function that explains the data is a constant with very high probability.

For inference about the complex data set, we use more computation. You can experiment with different amounts of computation to see how the quality of the inferences degrade with less computation. Note that we are using a very simple generic inference algorithm in this tutorial. In later tutorials, we will learn how to write more efficient algorithms, so that accurate results can be obtained with significantly less amount of computation.

In [ ]:
traces = [do_inference(changepoint_model, xs_dense, ys_complex, 100000) for _=1:12];
grid(render_cp_model_trace, traces)

------
### Exercise
Write a function that plots the histogram of the probability distribution on the number of changepoints.
Show the results for the `ys_simple` and `ys_complex` data sets.

-------

### Exercise
Write a new version of `changepoint_model` that uses `@splice` to make the recursive calls instead of `@addr`.

Hint: Recall that addresses can be arbitrary values, not just symbols.